In [2]:
pip install gdown


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lenovo\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [3]:
import gdown

url = "https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu"
output = "tennis_court_det_dataset.zip"
gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu
From (redirected): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&confirm=t&uuid=559b1be2-9a11-4f03-8cb5-0b1c852a2dee
To: C:\Official\PyCharm\tennis-analysis\training\tennis_court_det_dataset.zip
 65%|█████████████████████████████████████████████████████████████████████████████████████████████▊                                                   | 4.69G/7.26G [16:05<08:46, 4.86MB/s]


'tennis_court_det_dataset.zip'

In [7]:
import zipfile

with zipfile.ZipFile('tennis_court_det_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('tennis_court_det_dataset')  # This will extract into a folder


BadZipFile: File is not a zip file

In [ ]:
!pip install torch torchvision opencv-python numpy


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [ ]:
devic = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(items['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [ ]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [ ]:
model = model.to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

In [ ]:
torch.save(model.stat_dict(), "keypoints_model.pth")